In [8]:
import pandas as pd 
import numpy as np
import re

In [9]:
data = pd.read_csv("subjects-questions.csv")
data.head()

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,Biology
1,"Among the following organic acids, the acid pr...",Chemistry
2,If the area of two similar triangles are equal...,Maths
3,"In recent year, there has been a growing\nconc...",Biology
4,Which of the following statement\nregarding tr...,Physics


In [10]:
# Cleaning the data

In [11]:
subject_dictionary = {"Biology":1,"Chemistry":2,"Maths":3,"Physics":4}
data['Subject'] = data['Subject'].map(subject_dictionary)
data.Subject.unique()

array([1, 2, 3, 4])

In [12]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
def text_cleaning():
    data['eng'] = data['eng'].str.lower()
    stop = stopwords.words('english')
    data['eng'] = data['eng'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    data['eng'] = data['eng'].str.replace(r'[^\w\s]+', '')
    data['eng'] = data['eng'].replace('[^a-zA-Z0-9 ]', '', regex=True)
    stemmer = PorterStemmer()
    data['eng'] = data['eng'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in stop]).lower())
    return data

In [14]:
text_cleaning()

/tmp/ipykernel_35374/645509519.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['eng'] = data['eng'].str.replace(r'[^\w\s]+', '')


,eng,Subject
0,antiforest measur afforest b select graze c cl...,1
1,among follow organ acid acid present rancid bu...,2
2,area two similar triangl equal equilater b iso...,3
3,recent year grow concern gradual increas avera...,1
4,follow statement regard transform incorrect tr...,4
...,...,...
122514,follow group charact present chordat stage lif...,1
122515,light year light emit sun one year b time take...,4
122516,member dipnoi nativ india b africa australia c...,1
122517,one averagelif half activ nuclei decay b less ...,4


In [15]:
# Creating the Logistic Regression model

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression

In [19]:
count_vec = CountVectorizer()
text = count_vec.fit_transform(data['eng'])
sub = data['Subject']


In [20]:
X_train, X_test, y_train, y_test = train_test_split(text, sub, test_size=0.35)


log = LogisticRegression()

model = log.fit(X_train, y_train)


print(classification_report(y_test, model.predict(X_test)))
print(confusion_matrix(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           1       0.92      0.87      0.89      4573
           2       0.90      0.92      0.91     13212
           3       0.95      0.97      0.96     11560
           4       0.93      0.91      0.92     13537

    accuracy                           0.92     42882
   macro avg       0.92      0.92      0.92     42882
weighted avg       0.92      0.92      0.92     42882

[[ 3974   428    63   108]
 [  256 12141   136   679]
 [   27   118 11206   209]
 [   74   760   381 12322]]


/home/luissantiago/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
